In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load cleaned 45-feature dataset from the notebook
df = pd.read_pickle("observations.pkl")

# Keep IDs only for grouping (not in X)
id_cols = ["Unnamed: 0", "hadm_id", "icustay_id", "subject_id"]
present_id_cols = [c for c in id_cols if c in df.columns]

# Label
y = df["90D_Mortality"].astype(int)

# Features X (drop label + IDs)
drop_cols = ["90D_Mortality"] + present_id_cols
X = df.drop(columns=drop_cols, errors="ignore")

# ----- Actions -----
# Fluids: 5 bins (0..4)
if "TotalInput" not in df.columns:
    raise ValueError("TotalInput not found for fluid binning.")
fluid_bin = pd.qcut(df["TotalInput"], q=5, labels=False, duplicates="drop").fillna(0).astype(int)

# Optional vasopressor: if present, make 5×5 action space
possible_vp_cols = ["Vasopressor", "VasoDose", "VP_Dose", "VasopressorDose"]
vp_col = next((c for c in possible_vp_cols if c in df.columns), None)

if vp_col is not None:
    vaso_bin = pd.qcut(df[vp_col], q=5, labels=False, duplicates="drop").fillna(0).astype(int)
    action_id = (fluid_bin + 5 * vaso_bin).astype(int)   # 0..24
    nbins = 25
else:
    action_id = fluid_bin.copy()                         # 0..4
    nbins = 5

# ----- Fill, scale, and make next-state -----
X = X.fillna(X.mean(numeric_only=True))
Xnext = X.copy()

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
Xnext_scaled = pd.DataFrame(scaler.transform(Xnext), columns=X.columns)

# ----- Extra trainer fields -----
bloc_num = df["hadm_id"].astype(int).to_numpy() if "hadm_id" in df.columns else np.arange(len(df), dtype=int)
SOFAS = df["SOFA"].to_numpy() if "SOFA" in df.columns else np.zeros(len(df), dtype=float)
next_action = action_id.copy()
done = np.zeros(len(df), dtype=int)

# ----- Split (80 / 10 / 10) -----
(
    X_temp, X_test,
    y_temp, y_test,
    Xnext_temp, Xnext_test,
    action_temp, action_test,
    next_action_temp, next_action_test,
    done_temp, done_test,
    bloc_temp, bloc_test,
    SOFAS_temp, SOFAS_test
) = train_test_split(
    X_scaled, y, Xnext_scaled, action_id, next_action, done, bloc_num, SOFAS,
    test_size=0.1, random_state=42
)

(
    X_train, X_val,
    y_train, y_val,
    Xnext_train, Xnext_val,
    action_train, action_val,
    next_action_train, next_action_val,
    done_train, done_val,
    bloc_train, bloc_val,
    SOFAS_train, SOFAS_val
) = train_test_split(
    X_temp, y_temp, Xnext_temp, action_temp, next_action_temp, done_temp, bloc_temp, SOFAS_temp,
    test_size=0.1111, random_state=42
)

# ----- Save -----
final_data = {
    # features
    "X_train": X_train, "X_val": X_val, "X_test": X_test,
    "Xnext_train": Xnext_train, "Xnext_val": Xnext_val, "Xnext_test": Xnext_test,

    # labels (reward proxy)
    "y_train": y_train, "y_val": y_val, "y_test": y_test,

    # actions + extras
    "Action_train": action_train, "Action_val": action_val, "Action_test": action_test,
    "NextAction_train": next_action_train, "NextAction_val": next_action_val, "NextAction_test": next_action_test,
    "Done_train": done_train, "Done_val": done_val, "Done_test": done_test,
    "Bloc_train": bloc_train, "Bloc_val": bloc_val, "Bloc_test": bloc_test,
    "SOFAS_train": SOFAS_train, "SOFAS_val": SOFAS_val, "SOFAS_test": SOFAS_test,

    # metadata
    "nbins": nbins,
    "has_vasopressor": vp_col is not None
}
pd.to_pickle(final_data, "requiredFile.pkl")

# ----- Simple expected return (terminal-only ±24) -----
terminal_reward = np.where(y.to_numpy() == 0, 24.0, -24.0)
expected_return = terminal_reward.mean()
survival_rate = (y == 0).mean()

print(f"Saved requiredFile.pkl (nbins={nbins}, vasopressor_col={vp_col}).")
print(f"Survival rate (pooled): {survival_rate:.4f}")
print(f"Expected return (±24 terminal): {expected_return:.2f}")
